<a href="https://colab.research.google.com/github/pchunter97/clasificador_noticias/blob/main/LSTM_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#División del conjunto de datos

In [1]:
#Conectar al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import label_binarize



Leer el archivo

In [3]:
data_preprocesed = np.load("/content/drive/MyDrive/FakesNews/dataset_preprocesado.npy")
data_cleaned = pd.read_csv("/content/drive/MyDrive/FakesNews/dataset_final.csv")


In [4]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9294 entries, 0 to 9293
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9294 non-null   int64 
 1   clase       9294 non-null   int64 
 2   texto       9294 non-null   object
dtypes: int64(2), object(1)
memory usage: 218.0+ KB


In [5]:
y_train_ohe = label_binarize(data_cleaned['clase'], classes=[0,1])

In [6]:
x_train,x_test,y_train, y_test = train_test_split(data_preprocesed,y_train_ohe,test_size=0.2,random_state=50)

In [28]:
print("X_train info",x_train.shape)
print("\n\n\nX_test info",x_test.shape)
print("\n\n\nY_train info",y_train.shape)
print("\n\n\nY_test info",y_test.shape)

X_train info (7435, 300)



X_test info (1859, 300)



Y_train info (7435, 1)



Y_test info (1859, 1)


#Implementación del modelo

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [171]:
from sklearn import metrics
from keras.backend import dropout

from keras.layers.core.spatial_dropout import SpatialDropout2D
maxFeatures = 50

network = Sequential()
network.add(Dense(100))
network.add(Activation('Softmax'))
network.add(Dense(50))
network.add(Dense(25))
network.add(Dense(10))
network.add(Dense(1))
#network.add(Dense(1))
#network.add(Activation('relu'))


In [172]:
network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [173]:
history = network.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2, verbose=1)

Epoch 1/10
60/60 [==============================] - 1s 6ms/step - loss: 0.8482 - acc: 0.5032 - val_loss: 0.6733 - val_acc: 0.5474
Epoch 2/10
60/60 [==============================] - 0s 5ms/step - loss: 0.6487 - acc: 0.6110 - val_loss: 0.6344 - val_acc: 0.6577
Epoch 3/10
60/60 [==============================] - 0s 5ms/step - loss: 0.5904 - acc: 0.7155 - val_loss: 0.5885 - val_acc: 0.6967
Epoch 4/10
60/60 [==============================] - 0s 4ms/step - loss: 0.5540 - acc: 0.7337 - val_loss: 0.5802 - val_acc: 0.7108
Epoch 5/10
60/60 [==============================] - 0s 4ms/step - loss: 0.5350 - acc: 0.7431 - val_loss: 0.5673 - val_acc: 0.7256
Epoch 6/10
60/60 [==============================] - 0s 4ms/step - loss: 0.5204 - acc: 0.7505 - val_loss: 0.5600 - val_acc: 0.7229
Epoch 7/10
60/60 [==============================] - 0s 4ms/step - loss: 0.5353 - acc: 0.7559 - val_loss: 0.5769 - val_acc: 0.7236
Epoch 8/10
60/60 [==============================] - 0s 4ms/step - loss: 0.5253 - acc: 0.75

In [174]:
pred = network.predict(x_test)

In [175]:
for i in range(len(pred)):
    if(pred[i] > 0.5):
        pred[i] = 1
    else:
        pred[i] = 0

In [176]:
from sklearn.metrics import f1_score, classification_report
f1_score(pred,y_test)

0.8037037037037037

In [177]:
new_report = classification_report(y_test,pred,target_names=['0','1'])
print(new_report)

              precision    recall  f1-score   support

           0       0.76      0.70      0.73       814
           1       0.78      0.83      0.80      1045

    accuracy                           0.77      1859
   macro avg       0.77      0.76      0.77      1859
weighted avg       0.77      0.77      0.77      1859



In [47]:
network.save("/content/drive/MyDrive/FakesNews/LSTM.h5")

In [180]:
#test 1
new = np.array(x_test[200]).reshape(1,-1)
print(network.predict(new))

[[0.7909777]]
